<a href="https://colab.research.google.com/github/alaudee/rf_model_credit_default_classification/blob/main/random_oti_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pygad as pygad

from sklearn.impute import SimpleImputer
from scipy import stats
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score

In [11]:
arquivo_dat = 'australian.dat'
arquivo_csv = 'australian.csv'

columns_names = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15']

df = pd.read_csv(arquivo_dat, sep='\s+', header = None, names = columns_names)

df.to_csv(arquivo_csv, index=False)

print(f"Arquivo '{arquivo_dat}' convertido para '{arquivo_csv}' com sucesso!")

base = pd.read_csv('australian.csv')

Arquivo 'australian.dat' convertido para 'australian.csv' com sucesso!


<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3538067993.py:6: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(arquivo_dat, sep='\s+', header = None, names = columns_names)


In [12]:
column_names = base.columns

imputer = SimpleImputer(missing_values = np.nan, strategy='median')

# Fit the imputer to the data and transform it
imputed_base = imputer.fit_transform(base)

imputed_base_df = pd.DataFrame(imputed_base, columns=column_names)

print("Original Data:\n", base)
print("\nImputed Data:\n", imputed_base_df)

Original Data:
      A1     A2      A3  A4  A5  A6     A7  A8  A9  A10  A11  A12  A13   A14  \
0     1  22.08  11.460   2   4   4  1.585   0   0    0    1    2  100  1213   
1     0  22.67   7.000   2   8   4  0.165   0   0    0    0    2  160     1   
2     0  29.58   1.750   1   4   4  1.250   0   0    0    1    2  280     1   
3     0  21.67  11.500   1   5   3  0.000   1   1   11    1    2    0     1   
4     1  20.17   8.170   2   6   4  1.960   1   1   14    0    2   60   159   
..   ..    ...     ...  ..  ..  ..    ...  ..  ..  ...  ...  ...  ...   ...   
685   1  31.57  10.500   2  14   4  6.500   1   0    0    0    2    0     1   
686   1  20.67   0.415   2   8   4  0.125   0   0    0    0    2    0    45   
687   0  18.83   9.540   2   6   4  0.085   1   0    0    0    2  100     1   
688   0  27.42  14.500   2  14   8  3.085   1   1    1    0    2  120    12   
689   1  41.00   0.040   2  10   4  0.040   0   1    1    0    1  560     1   

     A15  
0      0  
1      0  
2 

In [13]:
X = imputed_base_df.drop('A15', axis=1)
y = imputed_base_df['A15']

# Dividir os dados em treino e teste com 30% para teste (test_size = 0.3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

'''
Escalonamento dos dados
'''

# Inicializa scalers
scaler = StandardScaler()

# for col in X.columns:
#   X[col] = scaler_standard.fit_transform(X[[col]])

# X.head()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train

array([[-1.44781002,  0.00632013, -0.24436123, ...,  0.22447421,
         0.6394979 , -0.18392714],
       [ 0.69069836, -1.19228195,  1.05047674, ...,  0.22447421,
        -0.37014394, -0.18374945],
       [ 0.69069836, -0.16417438,  0.00464607, ...,  0.22447421,
         1.53695731, -0.17184367],
       ...,
       [-1.44781002, -0.55656877, -0.76727656, ...,  0.22447421,
        -0.14577909,  0.85738398],
       [ 0.69069836,  0.44326585, -0.54316999, ...,  0.22447421,
         0.19637731, -0.18392714],
       [ 0.69069836,  0.42184694, -0.79217729, ...,  0.22447421,
         0.30295062, -0.18339405]])

In [14]:
# SEED para manter a replicabilidade do algoritmo
# dict para armazenar os melhores hiperparametros p/ fold executada

SEED = 5
best_hyperparams_per_fold = {}

In [15]:
'''
Função para gerar um modelo do tipo RandomForestClassifier
'''

# macro para n instanciar muitas vezes
TEMP_CRITERION = ''
def create_rf_model(hyperparameters):
  # converte a solucao de float para inteiro
  solution = solution.astype(int)

  '''
  regra para poder variar entre valores para o hiperparâmetro
  critetion = ['gini', 'entropy'], dado que nativamente pelo
  atributo da classe pyGAD, gene_space não suporta tipo de dados
  string
  '''

  if solution[4] == 1:
    TEMP_CRITERION = 'gini'
  elif solution[4] == 2:
    TEMP_CRITERION = 'entropy'

  rf = RandomForestClassifier(n_estimators=hyperparameters[0], max_depth=hyperparameters[1], min_samples_split=hyperparameters[2], min_samples_leaf=hyperparameters[3], criterion=TEMP_CRITERION, random_state=SEED)

  return rf

In [16]:
'''
Função fitness para otimização por AG
'''

def fitness(ga_instance, solution, solution_idx):

  # from pdb import set_trace; set_trace()

  rf = create_rf_model(solution)
  # rf = RandomForestClassifier(n_estimators=solution[0], max_depth=solution[1], min_samples_split=solution[2], min_samples_leaf=solution[3], criterion=TEMP_CRITERION, random_state=42)

  rf.fit(X_inner_train[train_inner_index], y_inner_train[train_inner_index])

  y_pred = rf.predict(X_inner_train[test_inner_index])

  recall = recall_score(y_inner_train[test_inner_index], y_pred, average = 'weighted')

  return recall

In [17]:
def create_ga_class():
  ga_instance = pygad.GA(
    num_generations=50,
    num_parents_mating=4,
    fitness_func=fitness,
    sol_per_pop=20,
    num_genes=5,
    gene_space=[range(50, 201), range(10, 20), range(5, 51), range(5, 51), [1, 2]],
    parent_selection_type="tournament",
    crossover_type="uniform",
    mutation_type="random",
    mutation_percent_genes=15
    )

  return ga_instance

In [18]:
'''
Instanciação do modelo Random Forest sem otimização de hiperparâmetros
'''

rf_model = RandomForestClassifier(random_state = SEED)

kf = KFold(n_splits=10, random_state = SEED, shuffle = True)
kf_inner = KFold(n_splits = 2, random_state = SEED, shuffle = True)

for train_outer_index, test_outer_index in kf.split(X_train, y_train):
    print("TRAIN:", train_outer_index, "TEST:", test_outer_index)

    X_inner_train, X_inner_test = X_train[train_outer_index], X_train[test_outer_index]
    y_inner_train, y_inner_test = y_train[train_outer_index], y_train[test_outer_index]

    for train_inner_index, test_inner_index in kf_inner.split(X_inner_train, y_inner_train):
        print("TRAIN:", train_inner_index, "TEST:", test_inner_index)

        ga_instance = create_ga_class()

        ga_instance.run()

        solution, solution_fitness, solution_idx = ga_instance.best_solution()


# modelo = RandomForestClassifier(random_state = SEED, n_estimators = 10)

# #performing cross-validation
# cv_scores = cross_val_score(modelo, X_train, y_train, cv=k_fold, scoring='recall')
# print("Cross Validation Scores: ", cv_scores)
# print("Average CV Score: ", cv_scores.mean())
# print("Number of CV Scores used in Average: ", len(cv_scores))

# modelo.fit(X_train, y_train)

# y_predict = modelo.predict(X_test)
# y_predict

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  29  30  31  32  33  35  36  37
  38  39  41  42  44  45  46  47  48  49  51  52  53  54  55  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 133 134 135 136
 137 138 140 142 143 144 145 146 147 148 149 150 151 152 153 154 156 157
 158 159 160 161 163 164 165 166 167 168 169 170 171 172 173 174 175 176
 177 178 179 180 181 182 183 184 185 187 188 189 190 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 213 214 215
 216 217 218 219 220 221 224 225 226 227 228 229 230 231 232 233 235 236
 238 239 241 243 244 245 246 247 249 250 251 252 253 254 255 256 257 258
 259 260 261 262 264 265 266 267 268 269 270

KeyError: '[9, 11, 18, 23, 27, 36, 37, 38, 53, 65, 69, 73, 75, 77, 78, 81, 83, 89, 94, 101, 102, 104, 120, 124, 125, 126, 131, 134, 143, 147, 151, 154, 163, 164, 166, 168, 169, 170, 174, 176, 188, 190, 192, 194, 198, 204, 207, 214, 217, 218, 219, 220, 221, 229, 243, 250, 253, 254, 256, 258, 259, 267, 268, 271, 282, 283, 286, 291, 293, 295, 296, 300, 304, 306, 309, 311, 313, 328, 329, 336, 337, 343, 346, 347, 348, 351, 352, 353, 355, 357, 359, 361, 366, 367, 371, 375, 376, 378, 380, 381, 388, 389, 394, 404, 405, 407, 409, 411, 413, 414, 428, 429, 430, 431, 432, 436, 439, 443, 446, 457, 458, 462, 465, 467, 470, 473, 479] not in index'